In [102]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [103]:
data = pd.read_csv('/content/drive/MyDrive/Credit_card_attrition/Card_Attrition.csv', sep=';')
print(data)

       CLIENTNUM  Attrition_Flag  ...  Trans_Count_Feb13 Trans_Count_Mar13
0      768805383               0  ...                  7                 7
1      818770008               0  ...                  7                 8
2      713982108               0  ...                  4                 5
3      769911858               0  ...                  2                 7
4      709106358               0  ...                  5                 6
...          ...             ...  ...                ...               ...
10122  772366833               0  ...                 18                18
10123  710638233               1  ...                  9                10
10124  716506083               1  ...                  6                12
10125  717406983               1  ...                  2                 9
10126  714337233               1  ...                  9                 9

[10127 rows x 27 columns]


### **DATA PROCESSING**

First, we need to check if there are any duplicate rows in the dataset

In [104]:
duplicate = data[data.duplicated()]
print(duplicate)

Empty DataFrame
Columns: [CLIENTNUM, Attrition_Flag, Customer_Age, Gender, Dependent_count, Education_Level, Marital_Status, Income_Category, Card_Category, Months_on_book, Total_Relationship_Count, Months_Inactive_12_mon, Contacts_Count_12_mon, Credit_Limit, Total_Revolving_Bal, Trans_Amt_Oct12, Trans_Amt_Nov12, Trans_Amt_Dec12, Trans_Amt_Jan13, Trans_Amt_Feb13, Trans_Amt_Mar13, Trans_Count_Oct12, Trans_Count_Nov12, Trans_Count_Dec12, Trans_Count_Jan13, Trans_Count_Feb13, Trans_Count_Mar13]
Index: []


So, there are no duplicate rows

Now, we need to check for any missing data in the whole dataset

In [105]:
data.isnull().sum()

CLIENTNUM                    0
Attrition_Flag               0
Customer_Age                14
Gender                       0
Dependent_count              0
Education_Level              0
Marital_Status               0
Income_Category              0
Card_Category                0
Months_on_book               0
Total_Relationship_Count     0
Months_Inactive_12_mon       0
Contacts_Count_12_mon        0
Credit_Limit                 4
Total_Revolving_Bal          0
Trans_Amt_Oct12              0
Trans_Amt_Nov12              0
Trans_Amt_Dec12              0
Trans_Amt_Jan13              0
Trans_Amt_Feb13              0
Trans_Amt_Mar13              0
Trans_Count_Oct12            0
Trans_Count_Nov12            0
Trans_Count_Dec12            0
Trans_Count_Jan13            0
Trans_Count_Feb13            0
Trans_Count_Mar13            0
dtype: int64

So, there are 14 empty values in Customer_Age and 4 empty values in Credit_Limit

We can fill these empty values with their coressponding class' median

In [106]:
data['Customer_Age'].fillna(data['Customer_Age'].median(), inplace= True)
data['Credit_Limit'].fillna(data['Credit_Limit'].median(), inplace= True)

In [107]:
data.isnull().sum()

CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Trans_Amt_Oct12             0
Trans_Amt_Nov12             0
Trans_Amt_Dec12             0
Trans_Amt_Jan13             0
Trans_Amt_Feb13             0
Trans_Amt_Mar13             0
Trans_Count_Oct12           0
Trans_Count_Nov12           0
Trans_Count_Dec12           0
Trans_Count_Jan13           0
Trans_Count_Feb13           0
Trans_Count_Mar13           0
dtype: int64

Missing Values successfully filled

Now, we have the data of "Transaction_count" and "Transaction_Amount" for each month

Handling these data in month is too much so we can combine them and handle them in quarters.

So, let's combine three months into one for both "Trans_Amt" as well as "Trans_count"

In [108]:
#Dividing 6 months of Transaction Amount data into 2 Quarters and combining them
data['Trans_Amt_Q1'] = data['Trans_Amt_Oct12'] + data['Trans_Amt_Nov12'] + data['Trans_Amt_Dec12']
data['Trans_Amt_Q2'] = data['Trans_Amt_Jan13'] + data['Trans_Amt_Feb13'] + data['Trans_Amt_Mar13']

#Dividing 6 months of Transaction Count data into 2 Quarters and combining them
data['Trans_Count_Q1'] = data['Trans_Count_Oct12'] + data['Trans_Count_Nov12'] + data['Trans_Count_Dec12']
data['Trans_Count_Q2'] = data['Trans_Count_Jan13'] + data['Trans_Count_Feb13'] + data['Trans_Count_Mar13']

data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Trans_Amt_Oct12,Trans_Amt_Nov12,Trans_Amt_Dec12,Trans_Amt_Jan13,Trans_Amt_Feb13,Trans_Amt_Mar13,Trans_Count_Oct12,Trans_Count_Nov12,Trans_Count_Dec12,Trans_Count_Jan13,Trans_Count_Feb13,Trans_Count_Mar13,Trans_Amt_Q1,Trans_Amt_Q2,Trans_Count_Q1,Trans_Count_Q2
0,768805383,0,45.0,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,0,237,253,196,196,262,0,7,9,12,7,7,490,654,16,26
1,818770008,0,49.0,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,0,266,242,323,222,238,0,1,6,11,7,8,508,783,7,26
2,713982108,0,51.0,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,0,195,330,967,213,182,0,1,5,5,4,5,525,1362,6,14
3,769911858,0,40.0,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2661,0,321,166,184,285,215,0,4,2,5,2,7,487,684,6,14
4,709106358,0,40.0,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,0,108,149,192,88,279,0,1,7,9,5,6,257,559,8,20


Noe, in this project, we are supposed to find the people who will leave the credit card after some time.

So, the ratio of people in the second quarter to people in first quarter can be a good tool to predict the upcoming future

In [109]:
data['Trans_Amt_ratio'] = data['Trans_Amt_Q2'] / data['Trans_Amt_Q1']

data['Trans_Count_ratio'] = data['Trans_Count_Q2'] / data['Trans_Count_Q1']

In [110]:
data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Trans_Amt_Oct12,Trans_Amt_Nov12,Trans_Amt_Dec12,Trans_Amt_Jan13,Trans_Amt_Feb13,Trans_Amt_Mar13,Trans_Count_Oct12,Trans_Count_Nov12,Trans_Count_Dec12,Trans_Count_Jan13,Trans_Count_Feb13,Trans_Count_Mar13,Trans_Amt_Q1,Trans_Amt_Q2,Trans_Count_Q1,Trans_Count_Q2,Trans_Amt_ratio,Trans_Count_ratio
0,768805383,0,45.0,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,0,237,253,196,196,262,0,7,9,12,7,7,490,654,16,26,1.334694,1.625000
1,818770008,0,49.0,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,0,266,242,323,222,238,0,1,6,11,7,8,508,783,7,26,1.541339,3.714286
2,713982108,0,51.0,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,0,195,330,967,213,182,0,1,5,5,4,5,525,1362,6,14,2.594286,2.333333
3,769911858,0,40.0,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2661,0,321,166,184,285,215,0,4,2,5,2,7,487,684,6,14,1.404517,2.333333
4,709106358,0,40.0,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,0,108,149,192,88,279,0,1,7,9,5,6,257,559,8,20,2.175097,2.500000


Now, we can drop the data of all 6 months and the newly formed quarter data

In [111]:
dropping = ['Trans_Amt_Oct12','Trans_Amt_Nov12', 'Trans_Amt_Dec12', 'Trans_Amt_Jan13', 'Trans_Amt_Feb13', 'Trans_Amt_Mar13',
            'Trans_Count_Oct12','Trans_Count_Nov12', 'Trans_Count_Dec12', 'Trans_Count_Jan13', 'Trans_Count_Feb13', 'Trans_Count_Mar13',
            'Trans_Count_Q1', 'Trans_Count_Q2', 'Trans_Amt_Q1', 'Trans_Amt_Q2','Education_Level', 'Income_Category', 'Customer_Age', 'CLIENTNUM']
data_new = data.drop(dropping, axis=1)
data_new.head()

,Attrition_Flag,Gender,Dependent_count,Marital_Status,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Trans_Amt_ratio,Trans_Count_ratio
0,0,M,3,Married,Blue,39,5,1,3,12691.0,777,1.334694,1.625000
1,0,F,5,Single,Blue,44,6,1,2,8256.0,864,1.541339,3.714286
2,0,M,3,Married,Blue,36,4,1,0,3418.0,0,2.594286,2.333333
3,0,F,4,Unknown,Blue,34,3,4,1,3313.0,2661,1.404517,2.333333
4,0,M,3,Married,Blue,21,5,1,0,4716.0,0,2.175097,2.500000


### **Numerical values to the categorical data**

In [112]:
data['Marital_Status'].value_counts()

Married     4687
Single      3943
Unknown      749
Divorced     748
Name: Marital_Status, dtype: int64

In [113]:
Marital_status_mapping = {"Married": 0, "Single": 1, "Unknown":2, "Divorced":3}
#for dataset in data_new:
data_new['Marital_Status'] = data_new['Marital_Status'].map(Marital_status_mapping)
data_new.head()

,Attrition_Flag,Gender,Dependent_count,Marital_Status,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Trans_Amt_ratio,Trans_Count_ratio
0,0,M,3,0,Blue,39,5,1,3,12691.0,777,1.334694,1.625000
1,0,F,5,1,Blue,44,6,1,2,8256.0,864,1.541339,3.714286
2,0,M,3,0,Blue,36,4,1,0,3418.0,0,2.594286,2.333333
3,0,F,4,2,Blue,34,3,4,1,3313.0,2661,1.404517,2.333333
4,0,M,3,0,Blue,21,5,1,0,4716.0,0,2.175097,2.500000


In [114]:
data['Gender'].value_counts()

F    5358
M    4769
Name: Gender, dtype: int64

In [115]:
sex_mapping = {"M": 0, "F": 1}
data_new['Gender'] = data_new['Gender'].map(sex_mapping)
data_new['Gender'].head()

0    0
1    1
2    0
3    1
4    0
Name: Gender, dtype: int64

In [116]:
data['Card_Category'].value_counts()

Blue        9436
Silver       555
Gold         116
Platinum      20
Name: Card_Category, dtype: int64

In [117]:
card_mapping = {"Blue": 0, "Silver": 1, "Gold":2, "Platinum":3}
data_new['Card_Category'] = data_new['Card_Category'].map(card_mapping)
data_new['Card_Category'].head()

0    0
1    0
2    0
3    0
4    0
Name: Card_Category, dtype: int64

In [118]:
data_new.head()

,Attrition_Flag,Gender,Dependent_count,Marital_Status,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Trans_Amt_ratio,Trans_Count_ratio
0,0,0,3,0,0,39,5,1,3,12691.0,777,1.334694,1.625000
1,0,1,5,1,0,44,6,1,2,8256.0,864,1.541339,3.714286
2,0,0,3,0,0,36,4,1,0,3418.0,0,2.594286,2.333333
3,0,1,4,2,0,34,3,4,1,3313.0,2661,1.404517,2.333333
4,0,0,3,0,0,21,5,1,0,4716.0,0,2.175097,2.500000


### **SPLITTING TEST AND TRAIN DATA**

In the given dataset, there is no test dataset so we will have to partition our data into train data and test data

Train_data: 7000
Test_data : 3127

In [119]:
x_train, x_test = train_test_split(data_new, train_size=7127, test_size=3000)
x_test.head()

,Attrition_Flag,Gender,Dependent_count,Marital_Status,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Trans_Amt_ratio,Trans_Count_ratio
5715,0,0,1,0,0,43,4,5,3,20958.0,0,0.768585,0.695652
6170,0,1,5,1,0,34,4,3,2,5950.0,0,0.439839,0.783784
9733,1,1,2,2,1,29,5,2,6,12203.0,0,0.900133,0.843137
8343,0,1,3,0,0,36,1,2,3,3297.0,1554,0.605263,0.720000
2181,0,0,1,1,0,43,4,2,3,11909.0,1018,1.006211,0.656250


In [123]:
y_train = x_train['Attrition_Flag']
x_train.drop('Attrition_Flag', axis=1, inplace=True)
print(x_train.shape)
print(y_train.shape)

(7127, 12)
(7127,)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [125]:
y_train.head()

9684     0
1100     0
10008    1
1601     0
9464     1
Name: Attrition_Flag, dtype: int64

In [126]:
y_test = x_test['Attrition_Flag']
x_test.drop('Attrition_Flag', axis=1, inplace=True)
print(x_test.shape)
print(y_test.shape)

(3000, 12)
(3000,)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [127]:
x_test.head()

,Gender,Dependent_count,Marital_Status,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Trans_Amt_ratio,Trans_Count_ratio
5715,0,1,0,0,43,4,5,3,20958.0,0,0.768585,0.695652
6170,1,5,1,0,34,4,3,2,5950.0,0,0.439839,0.783784
9733,1,2,2,1,29,5,2,6,12203.0,0,0.900133,0.843137
8343,1,3,0,0,36,1,2,3,3297.0,1554,0.605263,0.720000
2181,0,1,1,0,43,4,2,3,11909.0,1018,1.006211,0.656250


In [135]:
# Importing Classifier Modules
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Random Forest

In [139]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_train, y_train)
y_pred_rf = clf.predict(x_test)
accuracy_rf = accuracy_score(y_pred_rf, y_test)
print(accuracy_rf*100)

90.76666666666667


Decision Tree

In [140]:
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
y_pred_dt = clf.predict(x_test)
accuracy_dt = accuracy_score(y_pred_dt, y_test)
print(accuracy_dt*100)

86.63333333333333


So, Random Forest got the highest accuracy amongst the two